# Riot API로 받지 못하는 정보 OP.GG에서 크롤링하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### 각 라인의 탑 티어 챔피언 구하기

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
url = 'https://www.op.gg/champion/statistics'
html = requests.get(url, headers = headers).text
soup = BeautifulSoup(html, 'html.parser')

- 필요한 함수들

In [3]:
# 챔피언, 아이템, 스펠, 룬 데이터
champion_df = pd.read_csv('../00. data/champions.csv', index_col = 0)
champion_df['lower_name'] = champion_df['eng_name'].apply(lambda r : r.lower()) # OP.GG의 챔피언 찾기를 위해 소문자화
item_df = pd.read_csv('../00. data/items.csv', index_col = 0)
spell_df = pd.read_csv('../00. data/spell.csv', index_col = 0)
rune_df = pd.read_csv('../00. data/runes.csv', index_col = 0)

In [4]:
# 챔피언의 영문이름을 한글로 바꿔주는 함수
def translation_champion(eng):
    try :
        if eng == 'Wukong' :
            champion = '오공'
        elif eng == 'Nunu &amp; Willump':
            champion = '누누와 월럼프'
        else :
            eng = eng.lower().replace(' ','').replace('.','')
            champion = champion_df['name'][champion_df['name'][champion_df['lower_name'] == eng].index[0]]
    except:
        champion = eng 
    return champion

In [5]:
# 아이템키로 아이템 이름 불러오는 함수
def searchItem(itemKey):
    item = item_df['name'][item_df['name'][item_df['key'] == itemKey].index[0]]
    return item 

In [6]:
# 영어이름의 스펠을 한글로 바꿔주는 함수
def translation_spell(eng):
    spell = spell_df['name'][spell_df['name'][spell_df['eng'] == eng].index[0]]
    return spell

In [7]:
# 룬키로 룬 이름 불러오는 함수
def searchRune(runeKey):
    rune = rune_df['name'][rune_df['name'][rune_df['key'] == runeKey].index[0]]
    return rune

In [8]:
# 태그 지우는 함수
def removeTags(strings):
    strings = re.sub('<.+?>', '', str(strings), 0).strip().replace('[','').replace(']','')
    return strings

In [9]:
# OP.GG 아이템이미지의 키를 구하는 함수 + 룬
def findItem(itemLink):
    items = re.findall(r'\d\d\d\d.png', str(itemLink))
    items = [item.strip('.png') for item in items]
    
    return items

In [10]:
# OP.GG 스킬이미지의 영문 이름을 구하는 함수
def findSkill(link):
    name = re.findall(r'spell/.*png', str(link))
    name = ''.join(name).strip('spell/').strip('.png')
    return name

In [11]:
lanes = ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']

In [12]:
for lane in lanes:
    top10 = soup.find(attrs={'class':f'tabItem champion-trend-tier-{lane}'}).select('.champion-index-table__name')[:10] # 10개만 가져오기
    top10 = removeTags(top10).replace("'",'')
    top10 = top10.split(', ')
    kor_champions = [translation_champion(champ) for champ in top10]
    kor_champions = ', '.join(kor_champions)
    print(f'{lane}의 최고티어 챔피언:',kor_champions)

TOP의 최고티어 챔피언: 피오라, 카밀, 세트, 아트록스, 뽀삐, 이렐리아, 말파이트, 탐 켄치, 트린다미어, 오공
JUNGLE의 최고티어 챔피언: 자르반 4세, 리 신, 탈론, 탈리야, 니달리, 그레이브즈, 렉사이, 에코, 피들스틱, 킨드레드
MID의 최고티어 챔피언: 탈론, 르블랑, 카타리나, 아크샨, 벡스, 제드, 애니비아, 야스오, 신지드, 요네
ADC의 최고티어 챔피언: 미스 포츈, 베인, 직스, 이즈리얼, 사미라, 애쉬, 루시안, 진, 드레이븐, 카이사
SUPPORT의 최고티어 챔피언: 아무무, 블리츠크랭크, 레오나, 룰루, 쓰레쉬, 마오카이, 제라스, 파이크, 소라카, 노틸러스


### 원하는 챔피언의 기록 가져오기

In [13]:
# 샘플은 탑라인의 이렐리아
lane = 'top'
champion ='irelia'
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}'
html = requests.get(url, headers = headers).text
soup = BeautifulSoup(html, 'html.parser')

In [14]:
win_rank = soup.select('.champion-stats-trend-rank')[0].find('b').text + soup.select('.champion-stats-trend-rank')[0].find('span').text
pick_rank = soup.select('.champion-stats-trend-rank')[1].find('b').text + soup.select('.champion-stats-trend-rank')[1].find('span').text
print('승률 순위:', win_rank, '픽률 순위:', pick_rank)

승률 순위: 47/55 픽률 순위: 2/55


### 원하는 챔피언의 추천 세팅 

- 스펠 , 스킬트리, 아이템 트리

In [15]:
for i in range(2):
    tmp = soup.select('ul.champion-stats__list')[i].find_all('img')[0] 
    tmp2 = soup.select('ul.champion-stats__list')[i].find_all('img')[1]
    pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick')[i].find('strong').text
    pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick')[i].find('span').text
    winRate = soup.select('.champion-overview__stats.champion-overview__stats--win')[i].find('strong').text
    print('추천 스펠 : '+ translation_spell(findSkill(tmp)), translation_spell(findSkill(tmp2)), ' | 픽률 : '+ pickRate, '총표본 수 : ' + pickTotal, '승률 : '+ winRate)
skillSeq = []
for i in range(3):
    skill = findSkill(soup.select('ul.champion-stats__list')[2].find_all('img')[i*2])
    skillSeq.append(skill.lower().split(champion)[1])
pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick')[2].find('strong').text
pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick')[2].find('span').text
winRate = soup.select('.champion-overview__stats.champion-overview__stats--win')[2].find('strong').text
print('스킬 트리: ',' -> '.join(skillSeq).upper(),' | 픽률:', pickRate, '총표본 수 : ' + pickTotal, '승률:', winRate)    
for i in range(10):
    tmp = soup.select('td.champion-overview__data.champion-overview__border.champion-overview__border--first')[i].find_all('img')
    keys = findItem(tmp)
    items = [searchItem(int(key)) for key in keys]
    items = ', '.join(items)
    pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick.champion-overview__border')[i].find('strong').text
    pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick.champion-overview__border')[i].find('span').text
    winRate = soup.select('.champion-overview__stats.champion-overview__stats--win.champion-overview__border')[i].find('strong').text
    if i < 2:
        msg = '시작아이템 :'
    elif i >= 2 and i < 7:
        msg = '추천 빌드 :'
    else:
        msg = '신발 :'
    print(msg + items, '| 픽률 : '+ pickRate,'총표본 수 : ' + pickTotal, '승률 : '+ winRate)

추천 스펠 : 점멸 순간이동  | 픽률 : 77.72% 총표본 수 : 30,495 승률 : 48.2%
추천 스펠 : 점멸 점화  | 픽률 : 21.08% 총표본 수 : 8,272 승률 : 49.36%
스킬 트리:  Q -> W -> E  | 픽률: 41.55% 총표본 수 : 7,436 승률: 62.31%
시작아이템 :도란의 검, 체력 물약 | 픽률 : 82.78% 총표본 수 : 31,655 승률 : 49.13%
시작아이템 :부패 물약 | 픽률 : 8.87% 총표본 수 : 3,391 승률 : 44.88%
추천 빌드 :몰락한 왕의 검, 선혈포식자, 스테락의 도전 | 픽률 : 30.5% 총표본 수 : 5,040 승률 : 60.93%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 구인수의 격노검 | 픽률 : 10.97% 총표본 수 : 1,812 승률 : 64.02%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 마법사의 최후 | 픽률 : 8.23% 총표본 수 : 1,360 승률 : 63.01%
추천 빌드 :몰락한 왕의 검, 선혈포식자, 마법사의 최후 | 픽률 : 6.94% 총표본 수 : 1,146 승률 : 59.77%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 죽음의 무도 | 픽률 : 6.94% 총표본 수 : 1,146 승률 : 62.04%
신발 :판금 장화 | 픽률 : 68.66% 총표본 수 : 24,125 승률 : 49.91%
신발 :헤르메스의 발걸음 | 픽률 : 29.2% 총표본 수 : 10,261 승률 : 53.12%
신발 :명석함의 아이오니아 장화 | 픽률 : 1.37% 총표본 수 : 481 승률 : 67.36%


- 룬

In [16]:
mainRune = {'Precision' : '정밀', 'Domination' : '지배', 'Sorcery' : '마법', 'Resolve' : '결의', 'Inspiration' : '영감'}

In [17]:
# 첫번째 추천룬
total_info = soup.select_one('.tabItem.ChampionKeystoneRune-1')
rate_info = total_info.select('.champion-overview__stats.champion-overview__stats--pick')
rec_runes = soup.select('.champion-stats-summary-rune__name')
rune_info = soup.select('.champion-stats-summary-rune__rate')
eng_rune = rec_runes[0].text.split(' + ')
kor_rune = mainRune[eng_rune[0]] + ' + ' + mainRune[eng_rune[1]]
pickRate = rune_info[0].find('strong').text
winRate = rune_info[0].find_all('span')[-1].text
print(kor_rune, '| 픽률 : ' + pickRate, '승률 : ' + winRate)
for i in range(2): 
    pickRate = rate_info[i].find('strong').text
    winRate= rate_info[i].find_all('strong')[-1].text
    pickTotal = rate_info[i].find_all('span')[1].text
    print('추천룬 | 픽률 : ' + pickRate, '승률 : ' + winRate, '총 표본수 : ' + pickTotal)
    for k in range(33):
        if i == 1:
            k += 33
        link = total_info.find_all('img')[k].attrs['src']
        if 'e_grayscale' in str(link):
            pass
        else:
            print(searchRune(int(''.join(findItem(link)))))

정밀 + 결의 | 픽률 : 81.37% 승률 : 48.44%
추천룬 | 픽률 : 25.18% 승률 : 47.06% 총 표본수 : 9,878
정밀
정복자
승전보
전설: 민첩함
최후의 저항
결의
뼈 방패
소생
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9
추천룬 | 픽률 : 10.36% 승률 : 49.77% 총 표본수 : 4,063
정밀
정복자
승전보
전설: 민첩함
최후의 저항
결의
뼈 방패
불굴의 의지
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9


In [18]:
# 두번째 추천 룬
rate_info = soup.select_one('.ChampionKeystoneRune-2').select('.champion-overview__stats.champion-overview__stats--pick')
eng_rune = rec_runes[1].text.split(' + ')
kor_rune = mainRune[eng_rune[0]] + ' + ' + mainRune[eng_rune[1]]
pickRate = rune_info[1].find('strong').text
winRate = rune_info[1].find_all('span')[-1].text
print(kor_rune, '| 픽률 : ' + pickRate, '승률 : ' + winRate)
for i in range(2):
    pickRate = rate_info[i].find('strong').text
    winRate= rate_info[i].find_all('strong')[-1].text
    pickTotal = rate_info[i].find_all('span')[1].text
    print('추천룬 | 픽률 : ' + pickRate, '승률 : ' + winRate, '총 표본수 : ' + pickTotal)
    for k in range(33):
        if i == 1:
                k += 33
        link = soup.select_one('.ChampionKeystoneRune-2').find_all('img')[k].attrs['src']
        if 'e_grayscale' in str(link):
            pass
        else:
            print(searchRune(int(''.join(findItem(link)))))

정밀 + 영감 | 픽률 : 13.21% 승률 : 49.19%
추천룬 | 픽률 : 2% 승률 : 46.69% 총 표본수 : 786
정밀
정복자
승전보
전설: 민첩함
최후의 저항
영감
비스킷 배달
시간 왜곡 물약
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9
추천룬 | 픽률 : 1.33% 승률 : 52.58% 총 표본수 : 523
정밀
정복자
승전보
전설: 민첩함
최후의 저항
영감
비스킷 배달
우주적 통찰력
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9


### 매치업

In [19]:
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}/matchup'
html = requests.get(url, headers = headers).text
soup = BeautifulSoup(html, 'html.parser')

In [20]:
for i in range(50):
    champ = soup.select('.champion-matchup-list__champion')[i].find('span').text
    winRate = soup.select('.champion-matchup-list__champion')[i].find_all('span')[1].string
    winRate = [item.strip() for item in winRate if str(item)]
    winRate = ''.join(winRate)
    play_rate = soup.select('.champion-matchup-list__totalplayed')[i].find('span').text
    play_num = soup.select('.champion-matchup-list__totalplayed')[i].find('small').text
    print(f'{translation_champion(champion)} vs {translation_champion(champ)} 상대승률 :',winRate)
    print(f'통계량: {play_num} 통계 비율 {play_rate}')

이렐리아 vs 세트 상대승률 : 47.46%
통계량: 13,451 통계 비율 7.54%
이렐리아 vs 피오라 상대승률 : 46.54%
통계량: 11,911 통계 비율 6.68%
이렐리아 vs 아트록스 상대승률 : 52.74%
통계량: 8,007 통계 비율 4.49%
이렐리아 vs 카밀 상대승률 : 50.29%
통계량: 7,753 통계 비율 4.35%
이렐리아 vs 사일러스 상대승률 : 52.12%
통계량: 6,535 통계 비율 3.66%
이렐리아 vs 말파이트 상대승률 : 49.41%
통계량: 6,134 통계 비율 3.44%
이렐리아 vs 리븐 상대승률 : 48.22%
통계량: 5,987 통계 비율 3.36%
이렐리아 vs 다리우스 상대승률 : 51.73%
통계량: 5,908 통계 비율 3.31%
이렐리아 vs 잭스 상대승률 : 46.44%
통계량: 5,470 통계 비율 3.07%
이렐리아 vs 릴리아 상대승률 : 53%
통계량: 5,208 통계 비율 2.92%
이렐리아 vs 제이스 상대승률 : 56.7%
통계량: 5,146 통계 비율 2.88%
이렐리아 vs 가렌 상대승률 : 46.48%
통계량: 4,989 통계 비율 2.80%
이렐리아 vs 베인 상대승률 : 50.3%
통계량: 4,674 통계 비율 2.62%
이렐리아 vs 케넨 상대승률 : 53.67%
통계량: 4,654 통계 비율 2.61%
이렐리아 vs 트린다미어 상대승률 : 49.23%
통계량: 4,625 통계 비율 2.59%
이렐리아 vs 요네 상대승률 : 53.74%
통계량: 4,075 통계 비율 2.28%
이렐리아 vs 탐 켄치 상대승률 : 46.75%
통계량: 3,929 통계 비율 2.20%
이렐리아 vs 쉔 상대승률 : 47.38%
통계량: 3,702 통계 비율 2.07%
이렐리아 vs 티모 상대승률 : 50.41%
통계량: 3,509 통계 비율 1.97%
이렐리아 vs 모데카이저 상대승률 : 49.67%
통계량: 3,179 통계 비율 1.78%
이렐리아 vs 아칼리 상대승률 : 53.58%